In [56]:
import random
import string
import numpy as np

def get_random_string(length=28):
    # choose from all lowercase letter
    letters = string.ascii_letters
    return ''.join(random.choice(letters) for i in range(length))

# se ficar ruim, uma possível melhora é 
def add_random_asterisk(word):
    number_of_asterisks = random.randint(0,len(word))
    for asterisk in range(number_of_asterisks):
        random_idx = random.randint(0,len(word)-1)
        word_to_list = list(word)
        word_to_list[random_idx] = '*'
        word = "".join(word_to_list)
    return word
                          
def generate_pop(pop_size):
    pop = []
    for being in range(pop_size):
        pop.append(add_random_asterisk(get_random_string()))
    return pop

def word_to_integer_vector(word):
    return list(map(lambda x: ord(x),list(word)))

def evaluate_loss_function(word, target="METHINKS*IT*IS*LIKE*A*WEASEL"):
    print(f"Word:   {word}\n")
    print(f"Target: {target}\n")
    result = []
    int_word = word_to_integer_vector(word)
    int_target = word_to_integer_vector(target)
    for i in range(len(int_word)):
        result.append(np.abs(int_word[i] - int_target[i]))
    return np.sum(result)
    
word = generate_pop(1)[0]
evaluate_loss_function(word)

Word:   m**J*v**XKDa*Jk*T*M*X**G****

Target: METHINKS*IT*IS*LIKE*A*WEASEL



753

In [9]:
# gerar pop.

In [10]:
# converter de char para int (ascii)

In [6]:
# função de custo
# soma das diferenças entre os valores das duas strings

In [7]:
# mutação
# escolhe posição aleatória e troca char.

In [11]:
# selecionar os melhores indivíudos (ranking dos indivíduos)
# no caso, isso não vai ser feito
# cruzaremos todos os indivíduos
# se ficar ruim, tentamos outra abordagem: tipo ordear indivíduos e cruzar melhores entre si.

In [12]:
# cross (reprodução)
# recebe dois indivíduos e gera dois filhos 
# um ponto de corte aleatório